In [1]:
import math
from scipy.constants import Planck, alpha, e, epsilon_0, speed_of_light, m_p, m_n, c
from random import choices
import simpy as simp
import random as rm
import numpy as np

In [2]:
#function for caculating the electric dipole capture
def cap_E(l,R,v):
    M = 1
    h = (6.5821*(10**-16))/(10**6)
    Jl = (3*(R**l))/(l+3)
    ke = ((2*math.pi*v)/c)
    Qlm = ((3*(3.5*(10**-8)))**l)*e
    Qlmp = ((ke*h)/(M*c))*Qlm
    w = 2*math.pi*v
    E__amp = -((4*math.pi)/(math.factorial(math.factorial(2*l + 1))))*(((l+1)/l)**0.5)*(ke**(l+2))*Qlm
    sig_cap_lm = ((8*math.pi*(l+1))/(l*math.factorial(math.factorial(2*l + 1))**2))*((ke**(2*l + 1))/h)*(abs(Qlm+Qlmp))
    Ue = (c/(2*math.pi*(ke**2)))*(abs(E__amp)**2)
    return sig_cap_lm, Ue

#function for calculating the magnetic dipole caputure    
def cap_M(l,R,v):
    M = 1
    h = (6.5821*(10**-16))/(10**6)
    Jl = (3*(R**l))/(l+3)
    ke = ((2*math.pi*v)/c)
    Qlm = ((3*(3.5*(10**-8)))**l)*e
    Mlm = Qlm*(v/c)
    Mlmp = (math.sqrt((10*(h/(M*c*R))**2)*(abs(Qlm)**2)))+Mlm
    w = 2*math.pi*v
    M__amp = +((4*math.pi)/(math.factorial(math.factorial(2*l + 1))))*(((l+1)/l)**0.5)*(ke**(l+2))*Qlm
    sig_cap_lm = ((8*math.pi*(l+1))/(l*math.factorial(math.factorial(2*l + 1))**2))*((ke**(2*l + 1))/h)*(abs(Mlm+Mlmp))
    Um = (c/(2*math.pi*(ke**2)))*(abs(M__amp)**2)
    return sig_cap_lm, Um

def Tiab(l,v,A,e):
    M = A
    h = (6.5821*(10**-16))/(10**6)
    k = ((M*e)/(h**2))**(1/2)
    R = (A**(1/3))*(10**-13)
    ke = ((2*math.pi*v)/c)
    Qlm = ((3*(3.5*(10**-8)))**l)*e
    Mlm = Qlm*(v/c)
    Mlmp = (math.sqrt((10*(h/(M*c*R))**2)*(abs(Qlm)**2)))+Mlm
    Tm = ((8*math.pi*(l+1))/(l*math.factorial(math.factorial(2*l + 1))**2))*((ke**(2*l + 1))/h)*(abs(Mlm+Mlmp))**2
    a0 = (5.29177 * 10**-11)*100
    Z = A/2
    alph = 16*(l/(l+1))*((Z**3)/(a0**4))*((k**((2*l)-3))/(ke**((2*l)+1)))
    Tiab = alph*Tm
    return Tiab, Tm

In [3]:
counts = 5000
Type = 'hw+D'
def neutron_collisions(env,counts,Type):
    M = 1
    E = 0
    Me = 0
    scat = 0
    testNum = 0
    l = 1
    A = 2
    R = (A**(1/3))*(10**-13)
    h = (6.5821*(10**-16))/(10**6)
    #for the formation of a duetron by neutron, proton capture
    if Type=='n+p':
        for i in range(counts+1):
            print('Test %d' % testNum)
            #neutron energy is randomly chosen from the range 8.5*10^10 and 10^11
            e = rm.randint(1*(10**19),3*(10**19))
            yield env.timeout(1)
            k = ((M*e)/(h**2))**(1/2)
            v = (2*h*k)/M
            cap__M,UM = cap_M(l,R,v)
            react = ['cap M','scat']
            cross = [cap__M/(2*l + 1),1-((cap__M/(2*l + 1)))]
            #here the program takes a random weighted choice of the reactions using the probabilities that they will happen
            win=choices(react, cross)
            print(e/9.223e+18,'Kilo Joule')
            print(win)
            print(cross)
            if win==['scat']:
                scat+=1
            elif win==['cap M']:
                Me+=1
                print('The magnetic radiation emitted per second',UM)
            print(Me,scat,'\n')
            testNum+=1
    #for photodisitegration of the deutron
    elif Type=='hw+D':
        inter = 0
        Me = 0
        scat = 0
        proemis = 0
        testNum = 0
        for i in range(counts+1):
            M = 2
            A = 2
            l = 1
            h = (6.5821*(10**-16))/(10**6)
            R = (A**1/3)*(10**-13)
            #photon energy is randomly chosen from this range:
            print('Test',testNum)
            e = rm.randint(60,70)/(9*(10**10))
            k = ((M*e)/(h**2))**(1/2)
            v = (2*h*k)/M
            ke = ((2*math.pi*v)/c)
            cap__M,UM = cap_M(l,R,v)
            dis_M = cap__M*((k/ke)**2)
            Tiab_prob,Tm_prob = Tiab(l,v,A,e)
            react = ['dis M','com scattering','Internal Conversion','Emission of a Photon']
            cross = [dis_M/(2*l + 1),abs(1-(dis_M/(2*l +1))),Tiab_prob,Tm_prob]
            #here the program takes a random weighted choice of the reactions using the probabilities that they will happen
            win=choices(react, cross)
            print((e*1e+6),'eV')
            print(win)
            print(cross)
            if win==['dis M']:
                Me+=1
                print('The magnetic radiation emitted per second',UM)
            elif win==['com scattering']:
                scat+=1
            elif win==['Internal Conversion']:
                inter+=1
            else:
                proemis+=1
            print(Me,scat,proemis,inter,'\n')
            testNum+=1
        print('')
env = simp.Environment()
env.process(neutron_collisions(env,counts,Type))
env.run(until=counts+2)

Test 0
0.0007333333333333332 eV
['com scattering']
[0.5737175918278818, 0.42628240817211815, 9.99885852702315e-24, 2.9487648419842027e-77]
0 1 0 0 

Test 1
0.0006666666666666666 eV
['dis M']
[0.47415619973001943, 0.5258438002699806, 7.879039581937408e-24, 1.9203377887922856e-77]
The magnetic radiation emitted per second 2.8218835376677993e-96
1 1 0 0 

Test 2
0.0007444444444444444 eV
['com scattering']
[0.5912328762892743, 0.40876712371072565, 1.0381901160713558e-23, 3.15521051398818e-77]
1 2 0 0 

Test 3
0.0007333333333333332 eV
['dis M']
[0.5737175918278818, 0.42628240817211815, 9.99885852702315e-24, 2.9487648419842027e-77]
The magnetic radiation emitted per second 3.4144790805780396e-96
2 2 0 0 

Test 4
0.0007666666666666667 eV
['dis M']
[0.6270534873864803, 0.37294651261351974, 1.1174066531403192e-23, 3.60173104651493e-77]
The magnetic radiation emitted per second 3.7319409785656653e-96
3 2 0 0 

Test 5
0.0007333333333333332 eV
['com scattering']
[0.5737175918278818, 0.426282408172


Test 121
0.0007777777777777777 eV
['dis M']
[0.6453588134390477, 0.35464118656095234, 1.1583317735331143e-23, 3.842650576026653e-77]
The magnetic radiation emitted per second 3.840897037381171e-96
73 49 0 0 

Test 122
0.0007555555555555555 eV
['dis M']
[0.6090115082404205, 0.3909884917595795, 1.0773615672293856e-23, 3.3727266898899777e-77]
The magnetic radiation emitted per second 3.6245526328266403e-96
74 49 0 0 

Test 123
0.0007 eV
['dis M']
[0.5227518264323113, 0.47724817356768867, 8.901109885865169e-24, 2.3918123639096573e-77]
The magnetic radiation emitted per second 3.11112660027875e-96
75 49 0 0 

Test 124
0.0007555555555555555 eV
['dis M']
[0.6090115082404205, 0.3909884917595795, 1.0773615672293856e-23, 3.3727266898899777e-77]
The magnetic radiation emitted per second 3.6245526328266403e-96
76 49 0 0 

Test 125
0.0007777777777777777 eV
['com scattering']
[0.6453588134390477, 0.35464118656095234, 1.1583317735331143e-23, 3.842650576026653e-77]
76 50 0 0 

Test 126
0.000722222222

Test 218
0.0007777777777777777 eV
['dis M']
[0.6453588134390477, 0.35464118656095234, 1.1583317735331143e-23, 3.842650576026653e-77]
The magnetic radiation emitted per second 3.840897037381171e-96
128 91 0 0 

Test 219
0.0007 eV
['com scattering']
[0.5227518264323113, 0.47724817356768867, 8.901109885865169e-24, 2.3918123639096573e-77]
128 92 0 0 

Test 220
0.0007111111111111111 eV
['com scattering']
[0.5394770665873508, 0.46052293341264916, 9.258528588905792e-24, 2.5674604660723064e-77]
128 93 0 0 

Test 221
0.0006666666666666666 eV
['dis M']
[0.47415619973001943, 0.5258438002699806, 7.879039581937408e-24, 1.9203377887922856e-77]
The magnetic radiation emitted per second 2.8218835376677993e-96
129 93 0 0 

Test 222
0.0007555555555555555 eV
['com scattering']
[0.6090115082404205, 0.3909884917595795, 1.0773615672293856e-23, 3.3727266898899777e-77]
129 94 0 0 

Test 223
0.0007666666666666667 eV
['dis M']
[0.6270534873864803, 0.37294651261351974, 1.1174066531403192e-23, 3.60173104651493e-7

The magnetic radiation emitted per second 3.210676380635363e-96
202 141 0 0 

Test 343
0.0007 eV
['dis M']
[0.5227518264323113, 0.47724817356768867, 8.901109885865169e-24, 2.3918123639096573e-77]
The magnetic radiation emitted per second 3.11112660027875e-96
203 141 0 0 

Test 344
0.0007222222222222223 eV
['dis M']
[0.5564656551577575, 0.4435343448422425, 9.624422818617563e-24, 2.7529812436455745e-77]
The magnetic radiation emitted per second 3.3117938740684603e-96
204 141 0 0 

Test 345
0.0006777777777777778 eV
['com scattering']
[0.4900913926686685, 0.5099086073313315, 8.211432652706946e-24, 2.0686187196148235e-77]
204 142 0 0 

Test 346
0.0007777777777777777 eV
['com scattering']
[0.6453588134390477, 0.35464118656095234, 1.1583317735331143e-23, 3.842650576026653e-77]
204 143 0 0 

Test 347
0.0007222222222222223 eV
['com scattering']
[0.5564656551577575, 0.4435343448422425, 9.624422818617563e-24, 2.7529812436455745e-77]
204 144 0 0 

Test 348
0.0007 eV
['dis M']
[0.5227518264323113, 

Test 449
0.0007111111111111111 eV
['com scattering']
[0.5394770665873508, 0.46052293341264916, 9.258528588905792e-24, 2.5674604660723064e-77]
267 183 0 0 

Test 450
0.0007 eV
['com scattering']
[0.5227518264323113, 0.47724817356768867, 8.901109885865169e-24, 2.3918123639096573e-77]
267 184 0 0 

Test 451
0.0006666666666666666 eV
['dis M']
[0.47415619973001943, 0.5258438002699806, 7.879039581937408e-24, 1.9203377887922856e-77]
The magnetic radiation emitted per second 2.8218835376677993e-96
268 184 0 0 

Test 452
0.0007777777777777777 eV
['dis M']
[0.6453588134390477, 0.35464118656095234, 1.1583317735331143e-23, 3.842650576026653e-77]
The magnetic radiation emitted per second 3.840897037381171e-96
269 184 0 0 

Test 453
0.0006666666666666666 eV
['dis M']
[0.47415619973001943, 0.5258438002699806, 7.879039581937408e-24, 1.9203377887922856e-77]
The magnetic radiation emitted per second 2.8218835376677993e-96
270 184 0 0 

Test 454
0.000688888888888889 eV
['dis M']
[0.5062899350157758, 0.49

...

2562 1971 0 0 

Test 4533
0.0007222222222222223 eV
['dis M']
[0.5564656551577575, 0.4435343448422425, 9.624422818617563e-24, 2.7529812436455745e-77]
The magnetic radiation emitted per second 3.3117938740684603e-96
2563 1971 0 0 

Test 4534
0.0007555555555555555 eV
['dis M']
[0.6090115082404205, 0.3909884917595795, 1.0773615672293856e-23, 3.3727266898899777e-77]
The magnetic radiation emitted per second 3.6245526328266403e-96
2564 1971 0 0 

Test 4535
0.0007222222222222223 eV
['dis M']
[0.5564656551577575, 0.4435343448422425, 9.624422818617563e-24, 2.7529812436455745e-77]
The magnetic radiation emitted per second 3.3117938740684603e-96
2565 1971 0 0 

Test 4536
0.0007666666666666667 eV
['dis M']
[0.6270534873864803, 0.37294651261351974, 1.1174066531403192e-23, 3.60173104651493e-77]
The magnetic radiation emitted per second 3.7319409785656653e-96
2566 1971 0 0 

Test 4537
0.0007666666666666667 eV
['com scattering']
[0.6270534873864803, 0.37294651261351974, 1.1174066531403192e-23, 3.60173

2619 2023 0 0 

Test 4642
0.0007222222222222223 eV
['dis M']
[0.5564656551577575, 0.4435343448422425, 9.624422818617563e-24, 2.7529812436455745e-77]
The magnetic radiation emitted per second 3.3117938740684603e-96
2620 2023 0 0 

Test 4643
0.0007 eV
['dis M']
[0.5227518264323113, 0.47724817356768867, 8.901109885865169e-24, 2.3918123639096573e-77]
The magnetic radiation emitted per second 3.11112660027875e-96
2621 2023 0 0 

Test 4644
0.0007222222222222223 eV
['dis M']
[0.5564656551577575, 0.4435343448422425, 9.624422818617563e-24, 2.7529812436455745e-77]
The magnetic radiation emitted per second 3.3117938740684603e-96
2622 2023 0 0 

Test 4645
0.0007 eV
['dis M']
[0.5227518264323113, 0.47724817356768867, 8.901109885865169e-24, 2.3918123639096573e-77]
The magnetic radiation emitted per second 3.11112660027875e-96
2623 2023 0 0 

Test 4646
0.0007111111111111111 eV
['dis M']
[0.5394770665873508, 0.46052293341264916, 9.258528588905792e-24, 2.5674604660723064e-77]
The magnetic radiation emi

0.0007444444444444444 eV
['com scattering']
[0.5912328762892743, 0.40876712371072565, 1.0381901160713558e-23, 3.15521051398818e-77]
2675 2058 0 0 

Test 4733
0.0007444444444444444 eV
['dis M']
[0.5912328762892743, 0.40876712371072565, 1.0381901160713558e-23, 3.15521051398818e-77]
The magnetic radiation emitted per second 3.518732000164099e-96
2676 2058 0 0 

Test 4734
0.0007111111111111111 eV
['dis M']
[0.5394770665873508, 0.46052293341264916, 9.258528588905792e-24, 2.5674604660723064e-77]
The magnetic radiation emitted per second 3.210676380635363e-96
2677 2058 0 0 

Test 4735
0.000688888888888889 eV
['dis M']
[0.5062899350157758, 0.4937100649842242, 8.552100240146815e-24, 2.2256558290219734e-77]
The magnetic radiation emitted per second 3.013144532998618e-96
2678 2058 0 0 

Test 4736
0.0007777777777777777 eV
['com scattering']
[0.6453588134390477, 0.35464118656095234, 1.1583317735331143e-23, 3.842650576026653e-77]
2678 2059 0 0 

Test 4737
0.000688888888888889 eV
['com scattering']
[

2735 2090 0 0 

Test 4825
0.0007 eV
['com scattering']
[0.5227518264323113, 0.47724817356768867, 8.901109885865169e-24, 2.3918123639096573e-77]
2735 2091 0 0 

Test 4826
0.0007777777777777777 eV
['dis M']
[0.6453588134390477, 0.35464118656095234, 1.1583317735331143e-23, 3.842650576026653e-77]
The magnetic radiation emitted per second 3.840897037381171e-96
2736 2091 0 0 

Test 4827
0.0007666666666666667 eV
['com scattering']
[0.6270534873864803, 0.37294651261351974, 1.1174066531403192e-23, 3.60173104651493e-77]
2736 2092 0 0 

Test 4828
0.0007111111111111111 eV
['dis M']
[0.5394770665873508, 0.46052293341264916, 9.258528588905792e-24, 2.5674604660723064e-77]
The magnetic radiation emitted per second 3.210676380635363e-96
2737 2092 0 0 

Test 4829
0.0007222222222222223 eV
['dis M']
[0.5564656551577575, 0.4435343448422425, 9.624422818617563e-24, 2.7529812436455745e-77]
The magnetic radiation emitted per second 3.3117938740684603e-96
2738 2092 0 0 

Test 4830
0.0007333333333333332 eV
['com

The magnetic radiation emitted per second 2.91673017879497e-96
2817 2167 0 0 

Test 4984
0.0007111111111111111 eV
['dis M']
[0.5394770665873508, 0.46052293341264916, 9.258528588905792e-24, 2.5674604660723064e-77]
The magnetic radiation emitted per second 3.210676380635363e-96
2818 2167 0 0 

Test 4985
0.0007555555555555555 eV
['dis M']
[0.6090115082404205, 0.3909884917595795, 1.0773615672293856e-23, 3.3727266898899777e-77]
The magnetic radiation emitted per second 3.6245526328266403e-96
2819 2167 0 0 

Test 4986
0.0007444444444444444 eV
['dis M']
[0.5912328762892743, 0.40876712371072565, 1.0381901160713558e-23, 3.15521051398818e-77]
The magnetic radiation emitted per second 3.518732000164099e-96
2820 2167 0 0 

Test 4987
0.0007444444444444444 eV
['com scattering']
[0.5912328762892743, 0.40876712371072565, 1.0381901160713558e-23, 3.15521051398818e-77]
2820 2168 0 0 

Test 4988
0.0006666666666666666 eV
['dis M']
[0.47415619973001943, 0.5258438002699806, 7.879039581937408e-24, 1.920337788